In [ ]:
!pip install catboost

In [151]:
import pandas as pd
heartdata='s3://heartana/heart.csv'
df=pd.read_csv(heartdata)

In [152]:
df.shape

(303, 14)

In [153]:
df.columns

Index(['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh',
       'exng', 'oldpeak', 'slp', 'caa', 'thall', 'output'],
      dtype='object')

In [154]:
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [155]:
df.tail()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [156]:
df["output"].mean()

0.5445544554455446

In [157]:
X = df.drop('output', axis = 1)
y = df['output']

In [158]:
from sklearn.model_selection import train_test_split

x_tr,x_test,y_tr,y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [159]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

x_tr = scale.fit_transform(x_tr)
x_test = scale.transform(x_test)

In [161]:
scale.mean_

array([5.44628099e+01, 6.57024793e-01, 9.91735537e-01, 1.30359504e+02,
       2.46842975e+02, 1.28099174e-01, 5.53719008e-01, 1.50115702e+02,
       3.14049587e-01, 1.01322314e+00, 1.42148760e+00, 6.81818182e-01,
       2.30165289e+00])

In [162]:
scale.var_

array([8.43725838e+01, 2.25343214e-01, 1.04125401e+00, 2.82040178e+02,
       2.77584311e+03, 1.11689775e-01, 2.80172119e-01, 4.97565125e+02,
       2.15422444e-01, 1.21065159e+00, 3.67802746e-01, 9.77272727e-01,
       3.51154293e-01])

In [164]:
import math
(df.iloc[302][0]-5.44626066e+01)/math.sqrt(8.437276e+01)

0.2762400715979868

In [143]:
df.iloc[302]

age          57.0
sex           0.0
cp            1.0
trtbps      130.0
chol        236.0
fbs           0.0
restecg       0.0
thalachh    174.0
exng          0.0
oldpeak       0.0
slp           1.0
caa           1.0
thall         2.0
output        0.0
Name: 302, dtype: float64

In [165]:
df.iloc[302].values

array([ 57.,   0.,   1., 130., 236.,   0.,   0., 174.,   0.,   0.,   1.,
         1.,   2.,   0.])

In [166]:
scale.transform(df.iloc[302].values[:-1].reshape(1,-1))

array([[ 0.27621823, -1.38407465,  0.00809909, -0.02140663, -0.20580267,
        -0.38330071, -1.04610909,  1.07074858, -0.67663234, -0.92086403,
        -0.69498803,  0.32186034, -0.50904773]])

In [168]:
from tensorflow.keras import backend as K

from tensorflow.keras import layers, Sequential
from tensorflow.keras import activations

model = Sequential()
model.add(layers.Dense(1,input_shape=(13,)))
model.add(layers.Activation(activations.sigmoid))

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = model.fit(x_tr, y_tr, validation_data=(x_test, y_test), batch_size =4, epochs=50)

Train on 242 samples, validate on 61 samples
Epoch 1/50
242/242 [==============================] - 1s 3ms/sample - loss: 0.6346 - acc: 0.6529 - val_loss: 0.6680 - val_acc: 0.6885
Epoch 2/50
242/242 [==============================] - 0s 658us/sample - loss: 0.5973 - acc: 0.6983 - val_loss: 0.6282 - val_acc: 0.6885
Epoch 3/50
242/242 [==============================] - 0s 668us/sample - loss: 0.5666 - acc: 0.7273 - val_loss: 0.5951 - val_acc: 0.7377
Epoch 4/50
242/242 [==============================] - 0s 687us/sample - loss: 0.5404 - acc: 0.7397 - val_loss: 0.5676 - val_acc: 0.7541
Epoch 5/50
242/242 [==============================] - 0s 655us/sample - loss: 0.5178 - acc: 0.7438 - val_loss: 0.5444 - val_acc: 0.7869
Epoch 6/50
242/242 [==============================] - 0s 657us/sample - loss: 0.4985 - acc: 0.7603 - val_loss: 0.5234 - val_acc: 0.7869
Epoch 7/50
242/242 [==============================] - 0s 657us/sample - loss: 0.4819 - acc: 0.7645 - val_loss: 0.5047 - val_acc: 0.7869
Epoch

Observation: 
   - Keras model performs well with an accuracy of 88%. We will use this model to predict the heart attack.

In [169]:
weights = model.get_weights() 

In [175]:
weights

[array([[-0.09005275],
        [-0.7291416 ],
        [ 0.6598856 ],
        [-0.25411126],
        [-0.1614455 ],
        [ 0.1166103 ],
        [ 0.24034064],
        [ 0.46880907],
        [-0.49103144],
        [-0.7343311 ],
        [ 0.30284047],
        [-0.6067721 ],
        [-0.55872846]], dtype=float32),
 array([0.16516548], dtype=float32)]